In [57]:
import pandas as pd
import numpy as np
import sqlite3
import seaborn as sns

In [58]:
con = sqlite3.connect('dfr.db')
cur = con.cursor()

# Reading table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
print(table_list)

# Get tables into dataframes
post_df = pd.read_sql_query('SELECT * FROM post', con)
comment_df = pd.read_sql_query('SELECT * FROM comment', con)

con.close()

[('version',), ('reddit_object_list',), ('reddit_object',), ('download_session',), ('reddit_object_list_association',), ('user',), ('subreddit',), ('download_task',), ('post',), ('comment',), ('content',), ('alembic_version',)]


In [59]:
post_df.columns

Index(['id', 'title', 'date_posted', 'domain', 'score', 'nsfw', 'reddit_id',
       'url', 'is_self', 'text', 'text_html', 'extracted', 'extraction_date',
       'extraction_error', 'error_message', 'retry_attempts', 'author_id',
       'subreddit_id', 'significant_reddit_object_id', 'download_session_id'],
      dtype='object')

In [60]:
# Checks out to be unique
post_df.title.value_counts()

EECS 496                                                   3
Summer Sublease                                            2
LING 111                                                   2
U-M Uniqname & Account…without decision?                   1
LSA Internship Scholarship Availability?                   1
                                                          ..
How much time a day do you guys study?                     1
co-op security features                                    1
thinking about transferring but idk if i’m overreacting    1
Ah yes EECS Piazza Literature                              1
Boring campus                                              1
Name: title, Length: 1508, dtype: int64

# Basic Modifications

### * Remove irrelevant columns

In [61]:
post_df = post_df.drop(
    columns=['url',
             'score',
             'nsfw',
             'is_self',
             'domain',
             'reddit_id',
             'text_html',
             'extracted',
             'extraction_date',
             'extraction_error',
             'error_message',
             'retry_attempts',
             'subreddit_id',
             'significant_reddit_object_id',
             'download_session_id']
)

comment_df = comment_df.drop(
    columns=['body_html',
             'score',
             'date_added',
             'reddit_id',
             'extracted',
             'extraction_date',
             'has_content',
             'extraction_error',
             'error_message',
             'retry_attempts',
             'subreddit_id',
             'download_session_id'
             ]
)

### * Convert date time to timestamp format

In [62]:
post_df.date_posted = post_df.date_posted.apply(lambda x: pd.to_datetime(x[:10]))
comment_df.date_posted = comment_df.date_posted.apply(lambda x: pd.to_datetime(x[:10]))

In [63]:
print(post_df.date_posted.min())
print(post_df.date_posted.max())

2022-12-23 00:00:00
2023-03-05 00:00:00


In [64]:
print(comment_df.date_posted.min())
print(comment_df.date_posted.max())

2022-12-23 00:00:00
2023-03-05 00:00:00


In [65]:
post_counts = post_df.set_index('date_posted').resample('1W').title.count()

In [66]:
# The number of posts collected each week
post_counts

date_posted
2022-12-25     34
2023-01-01    114
2023-01-08    202
2023-01-15    164
2023-01-22    136
2023-01-29    167
2023-02-05    173
2023-02-12    140
2023-02-19    144
2023-02-26    139
2023-03-05     99
Freq: W-SUN, Name: title, dtype: int64

In [67]:
comment_counts = comment_df.set_index('date_posted').resample('1W').body.count()

In [68]:
# The number of comments collected each week
comment_counts

date_posted
2022-12-25     302
2023-01-01     943
2023-01-08    1021
2023-01-15    1205
2023-01-22    1287
2023-01-29    1500
2023-02-05    1615
2023-02-12    1394
2023-02-19    1463
2023-02-26    1410
2023-03-05    1047
Freq: W-SUN, Name: body, dtype: int64

### Posts have two text fields, 'title' and 'text', combine them

In [69]:
# Putting a period in between. 
# Assuming that having an extra punctuation (e.g. '?.') is less harmful than missing a punctuation.
post_df['title_text_combined'] = post_df[['title','text']].apply(lambda x : '{}. {}'.format(x[0],x[1]), axis=1)

### New columns for sentiment_label and entities

In [70]:
post_df = post_df.reindex(columns = post_df.columns.tolist() + ['sentiment_labels','entities'])
comment_df = comment_df.reindex(columns = comment_df.columns.tolist() + ['sentiment_labels','entities'])

In [71]:
post_df

,id,title,date_posted,text,author_id,title_text_combined,sentiment_labels,entities
0,1000,"W23 Math Advice (Math 465, 451, 472)",2022-12-24,Hello! I will be taking a bunch of math + CS c...,96,"W23 Math Advice (Math 465, 451, 472). Hello! I...",NaN,NaN
1,1001,"EECS 370, 270 and 281/301",2022-12-23,I'm EE thinking of doing embedded systems. I'm...,2389,"EECS 370, 270 and 281/301. I'm EE thinking of ...",NaN,NaN
2,1002,How important is physics 140 for 240?,2022-12-23,It's been a while since I touched physics 140 ...,592,How important is physics 140 for 240?. It's be...,NaN,NaN
3,1003,Classic Post-Exam EECS Piazza Moment,2022-12-23,None,2399,Classic Post-Exam EECS Piazza Moment. None,NaN,NaN
4,1004,got my first C,2022-12-23,we just got EECS 281 grades back and i just go...,2400,got my first C. we just got EECS 281 grades ba...,NaN,NaN
...,...,...,...,...,...,...,...,...
1507,2507,Switch Major School of Kinesiology,2023-02-06,I am currently a kines student. Am I allowed t...,3416,Switch Major School of Kinesiology. I am curre...,NaN,NaN
1508,2508,best spots to take a nap on central?,2023-02-06,i live on north campus so i stay on central fo...,3417,best spots to take a nap on central?. i live o...,NaN,NaN
1509,2509,Ross finance club rankings,2023-01-18,To those looking to navigate finance club recr...,3422,Ross finance club rankings. To those looking t...,NaN,NaN
1510,2510,Selling Airpods (2nd Gen),2023-01-16,"Brand new, box opened (bought for $136 at tech...",89,"Selling Airpods (2nd Gen). Brand new, box open...",NaN,NaN


In [72]:
comment_df

,id,body,date_posted,author_id,post_id,parent_id,sentiment_labels,entities
0,1000,Because not everybody treats every day like it...,2023-01-23,91,161,NaN,NaN,NaN
1,1001,I mean Im “alternative” and I dress in colors ...,2023-01-23,492,161,NaN,NaN,NaN
2,1002,Because U-M is a machine for inducing depression,2023-01-23,330,161,NaN,NaN,NaN
3,1003,"It’s a p all around color, it goes with p much...",2023-01-23,102,161,NaN,NaN,NaN
4,1004,Have you considered that everyone at Umich has...,2023-01-23,314,161,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
13182,17179,not this time of year you can't . unless you j...,2023-02-06,232,2508,17165.0,NaN,NaN
13183,17180,Grad Library had some very sleep-inducing carr...,2023-02-06,3421,2508,17166.0,NaN,NaN
13184,17181,do I get a talking cat too?,2023-02-06,765,2508,17171.0,NaN,NaN
13185,17182,I've seen them little shits under furniture an...,2023-02-10,2007,2508,17178.0,NaN,NaN


### parent_id needs to be integer to match the id column

In [73]:
comment_df.parent_id = comment_df.parent_id.fillna(-1).astype('int64')

In [74]:
comment_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13187 entries, 0 to 13186
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   id                13187 non-null  int64         
 1   body              13187 non-null  object        
 2   date_posted       13187 non-null  datetime64[ns]
 3   author_id         13187 non-null  int64         
 4   post_id           13187 non-null  int64         
 5   parent_id         13187 non-null  int64         
 6   sentiment_labels  0 non-null      float64       
 7   entities          0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(4), object(1)
memory usage: 824.3+ KB


In [75]:
post_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1512 entries, 0 to 1511
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   id                   1512 non-null   int64         
 1   title                1512 non-null   object        
 2   date_posted          1512 non-null   datetime64[ns]
 3   text                 1253 non-null   object        
 4   author_id            1512 non-null   int64         
 5   title_text_combined  1512 non-null   object        
 6   sentiment_labels     0 non-null      float64       
 7   entities             0 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 94.6+ KB


In [76]:
#post_df.to_csv('posts.csv', index=False)
#comment_df.to_csv('comments.csv', index=False)

### Samples for manual labeling

In [77]:
post_sample = post_df.sample(450)
comment_sample = comment_df.sample(450)

In [78]:
subsamples = np.array_split(post_sample, 3)

In [79]:
len(subsamples[0]) == len(subsamples[2]) 

True

In [80]:
#subsamples[0].to_csv('charlie_manual.csv', index=False)
#subsamples[1].to_csv('daniel_manual.csv', index=False)
#subsamples[2].to_csv('dillan_manual.csv', index=False)